In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests python-dotenv tika

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads
'''


'''
匯入套件
'''
# 網路請求工具
import requests as req

# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 取得錯誤訊息
import sys, traceback

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# SQLite 資料庫
import sqlite3

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 取得系統時間的工具
from datetime import datetime

# 引入 hashlib 模組
import hashlib

# 高階文件操作工具
import shutil

# 檔案剖析工具
from tika import parser

# 讀取 .env
from dotenv import load_dotenv

# 取得網路資源的工具
import urllib 

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent(cache=True) # cache=True 表示從已經儲存的列表中提取

In [ ]:
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument('--disable-gpu')
my_options.add_argument('--disable-software-rasterizer')
my_options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"')

# 給 web driver 用的變數
driver = None

# 來源首頁
root_url = 'https://ndltd.ncl.edu.tw'
prefix_url = root_url + ''
path_url = prefix_url + ''
url = path_url + ''

# 指定 sheet name
folderName = sheetName = 'ndltd_ncl_edu_tw'

# 指定 json 檔名
jsonFileName = f'{folderName}.json'

# 建立儲存檔案用的資料夾
folderPath = f'./{folderName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
# 設定 Chrome 下載路徑 (需要絕對路徑)
fullDownloadPath = os.getcwd() + '\\' + folderName

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": fullDownloadPath,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing_for_trusted_sources_enabled": False,
    "safebrowsing.enabled": False,
    "plugins.always_open_pdf_externally": True
})

# 請求標頭
my_headers = {
    'User-Agent': ua.random
}

# 放置爬取的資料
listData = []

In [ ]:
'''
函式
'''
# md5 (用來為每一筆資料建立唯一代號)
def md5(string):
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    return m.hexdigest()

# 初始化 Web Driver
def init():
    global driver
    # 使用 Chrome 的 WebDriver
    
    driver = webdriver.Chrome( 
        options = my_options, 
        service = Service(ChromeDriverManager().install())
    )
    
# 走訪頁面
def visit():
    global driver
    
    try:
        # 走訪首頁
        driver.get(url)

        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.user_area a[href][title="登入"]')
            )
        )
        
        # 點按登入連結
        driver.find_element(By.CSS_SELECTOR, 'div.user_area a[href][title="登入"]').click()
        
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'input[type="text"][name="userid"]')
            )
        )
        
        # 讀取登入資訊
        load_dotenv()
        EMAIL = os.getenv("EMAIL")
        PWD = os.getenv("PWD")

        # 輸入 e-mail
        driver.find_element(By.CSS_SELECTOR, 'input[type="text"][name="userid"]').send_keys(EMAIL)
        
        # 輸入 password
        driver.find_element(By.CSS_SELECTOR, 'input[type="password"][name="passwd"]').send_keys(EMAIL)

        
    except TimeoutException as e:
        print('等待逾時: visit')
        
# 下載驗證碼圖片
def download():
    global driver
    
    try:
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.dispcheckimg_div img[src]')
            )
        )
        
        # 過濾重複的圖片
        mySet = set()
        
        # 要抓幾張
        count = 0
        limit = 1000
    
        # 下載圖片
        while count < limit:
            # 等待目標元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'div.dispcheckimg_div img[src]')
                )
            )
            
            # 取得圖片元素
            img_captcha = driver.find_element(By.CSS_SELECTOR, 'div.dispcheckimg_div img[src]')
            
            # 確認圖片是否完整讀取
            if img_captcha.size['width'] == 0:
                continue
            
            # 取得圖片 base64，轉成 md5
            str_md5 = md5(img_captcha.screenshot_as_base64)
            
            # 若是該圖片先前沒有出現過，才進行下載
            if str_md5 not in mySet:
                # 儲存圖片
                img_captcha.screenshot(f'{fullDownloadPath}/{count}.png')
                
                # 將 md5 加到 set 中
                mySet.add(str_md5)
                
                # 計數
                count += 1

            # 更換圖片
            driver.find_element(By.CSS_SELECTOR, 'div.dispcheckimg_div a[href][title="更換驗證碼"]').click()  
            
    except TimeoutException as e:
        print('等待逾時: download()')

        
# 關閉瀏覽器
def close():
    global driver
    driver.quit()


In [ ]:
init()

In [ ]:
visit()

In [ ]:
time_begin = time.time()
download()
time_end = time.time()
print(f"總共執行了 { time_end - time_begin } 秒")

In [ ]:
close()